In [38]:
from json import dumps, loads
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from requests import post, get

# MEETUP

In [ ]:
start_date_range = datetime.now(ZoneInfo('America/New_York')).__sub__(timedelta(days=1)).replace(hour=23, minute=0, second=0)
end_date_range = start_date_range.__add__(timedelta(days=60)).replace(hour=23, minute=0, second=0)

group_query = dumps({
    'operationName': 'getRecommendedGroups', 
    'variables': {
        'first': 200, 'lat': '-23.530000686645508', 'lon': '-46.630001068115234', 
        'topicCategoryId': '546', 'radius': 31, 'indexAlias': 'popular_groups_nearby_current'
    },
    'extensions': {
        'persistedQuery': {'version': 1, 'sha256Hash': '2a95947caf3b4c2f961b505c94044dd0994c0e53234a6f21b971dc256d72160b'}
    }
})

event_query = dumps({
    'operationName': 'recommendedEventsWithSeries',
    'variables': {
        'first': 200, 'lat': '-23.530000686645508', 'lon': '-46.630001068115234',
        'startDateRange': start_date_range.isoformat(timespec='seconds'), 
        'endDateRange': end_date_range.isoformat(timespec='seconds'),
        'eventType': 'PHYSICAL', 'topicCategoryId': '546'
    }, 
    'extensions': {
        'persistedQuery': {'version': 1, 'sha256Hash': '4b0e4ad488864d476bd860c73ec1ac8fc5cd951b89c5cf6aeecf2cbb822c992d'}
    }
})

def fetch_data(query_type):
    body = loads(post('https://www.meetup.com/gql2', data=query_type).content)
    if query_type == group_query:
        return [
            edge.get('node') for edge in body.get('data').get('results').get('edges')
            if edge.get('node').get('stats').get('eventRatings').get('totalRatings') != 0
            and not edge.get('node').get('isNewGroup')
        ]
    elif query_type == event_query:
        return [edge.get('node') for edge in body.get('data').get('result').get('edges')]
    
def get_data(node, obj_type):
    if obj_type == 'event':
        return {
            'id': node.get('id'), 'name': node.get('title'), 
            'description': node.get('description'), 'eventUrl': node.get('eventUrl'), 
            'eventPhoto': node.get('featuredEventPhoto').get('highResUrl').replace('.jpeg', '.avif'),
            'dateTime': datetime.strptime(''.join(node.get('dateTime').rsplit(':', 1)), '%Y-%m-%dT%H:%M:%S%z'),
            'venue': {
                'address': node.get('venue').get('address'), 'city': node.get('venue').get('city'),
                'state': node.get('venue').get('state')
            }, 'groupID': node.get('group').get('id')
        }
    elif obj_type == 'group':
        return {
            'id': node.get('group').get('id'), 'name': node.get('group').get('name'), 
            'description': node.get('description'), 'groupUrl': node.get('link'), 'city': node.get('group').get('city'),
            'groupPhoto': f'{node.get("groupPhoto").get("baseUrl")}{node.get("groupPhoto").get("id")}.avif',
            'totalRatings': node.get('group').get('stats').get('eventRatings').get('totalRatings'),
            'averageRatings': node.get('group').get('stats').get('eventRatings').get('average')
        }

# GUILD

In [ ]:
# add others plataforms

NameError: name 'obj_type' is not defined